In [1]:
import torch

unseen_ind = [4]
seen_ind = [i for i in range(6) if i not in unseen_ind]

seen_weights = {}
for i in seen_ind:
    seen_weights[i] = torch.load(f'FT_Client{i}/lora/lora_weights.pt')
# Load the LoRA weights file
# lora0_weights = torch.load('FT_region0/lora/lora_weights.pt')
# lora1_weights = torch.load('FT_region1/lora/lora_weights.pt')
# lora2_weights = torch.load('FT_region2/lora/lora_weights.pt')
# lora3_weights = torch.load('FT_region3/lora/lora_weights.pt')
# lora4_weights = torch.load('FT_region4/lora/lora_weights.pt')
# lora5_weights = torch.load('FT_region5/lora/lora_weights.pt')
# lora6_weights = torch.load('FT_region6/lora/lora_weights.pt')

In [2]:
unseen_weights1 = {} # 2 source
unseen_weights2 = {} # 3 source
unseen_weights3 = {} # 4 source

for k in seen_weights[0].keys():
    unseen_weights1[k] = 0.5*seen_weights[2][k] + 0.5*seen_weights[3][k]
    unseen_weights2[k] = 0.3333*seen_weights[2][k] + 0.3333*seen_weights[3][k] + 0.3333*seen_weights[1][k]
    unseen_weights2[k] = 0.1*seen_weights[0][k] + 0.3*seen_weights[1][k] + 0.3*seen_weights[2][k] + 0.3*seen_weights[3][k]
    # unseen_weights3[k] = 0.053125*seen_weights[10][k] + 0.901875*seen_weights[11][k] + 0.015*seen_weights[12][k] + 0.0025*seen_weights[13][k]

torch.save(unseen_weights1, f'FT_Client{unseen_ind[0]}/lora/lora_weights_2source.pt')
torch.save(unseen_weights2, f'FT_Client{unseen_ind[0]}/lora/lora_weights_3source.pt')
torch.save(unseen_weights3, f'FT_Client{unseen_ind[0]}/lora/lora_weights_4source.pt')
# torch.save(unseen_weights2, f'FT_region{unseen_ind[1]}/lora/lora_weights.pt')
# torch.save(unseen_weights3, f'FT_region{unseen_ind[2]}/lora/lora_weights.pt')

In [2]:
print(seen_weights.keys())
print(len(seen_weights[0]))
for k in seen_weights[0].keys():
    print(k)
    print(seen_weights[0][k].shape)

dict_keys([0, 1, 2, 3, 5])
288
spatial_block.blocks.0.s_attn.w_q.lora_A_r
torch.Size([8, 128])
spatial_block.blocks.0.s_attn.w_q.lora_B_r
torch.Size([128, 8])
spatial_block.blocks.0.s_attn.w_q.lora_A_i
torch.Size([8, 128])
spatial_block.blocks.0.s_attn.w_q.lora_B_i
torch.Size([128, 8])
spatial_block.blocks.0.s_attn.w_k.lora_A_r
torch.Size([8, 128])
spatial_block.blocks.0.s_attn.w_k.lora_B_r
torch.Size([128, 8])
spatial_block.blocks.0.s_attn.w_k.lora_A_i
torch.Size([8, 128])
spatial_block.blocks.0.s_attn.w_k.lora_B_i
torch.Size([128, 8])
spatial_block.blocks.0.s_attn.w_v.lora_A_r
torch.Size([8, 128])
spatial_block.blocks.0.s_attn.w_v.lora_B_r
torch.Size([128, 8])
spatial_block.blocks.0.s_attn.w_v.lora_A_i
torch.Size([8, 128])
spatial_block.blocks.0.s_attn.w_v.lora_B_i
torch.Size([128, 8])
spatial_block.blocks.1.s_attn.w_q.lora_A_r
torch.Size([8, 128])
spatial_block.blocks.1.s_attn.w_q.lora_B_r
torch.Size([128, 8])
spatial_block.blocks.1.s_attn.w_q.lora_A_i
torch.Size([8, 128])
spatial_b

In [ ]:
# def showKmeans(seen_weights, idxs, block, qkv,ri):
#     keyA = f'blocks.{block}.attn.w_{qkv}.lora_A_{ri}'
#     keyB = f'blocks.{block}.attn.w_{qkv}.lora_B_{ri}'
#     weightsA = [seen_weights[i][keyA] for i in idxs]
#     weightsB = [seen_weights[i][keyB] for i in idxs]
#     weights = [torch.cat([weightsA[i].T, weightsB[i]], dim=0) for i in range(len(weightsA))]
#     weights = torch.cat(weights, dim=1)
#     print(weights.shape)
#     # 对weights进行kmeans聚类
#     from sklearn.cluster import KMeans
#     kmeans = KMeans(n_clusters=8, random_state=0).fit(weights.T.cpu().detach().numpy())
#     print(kmeans.labels_.shape)
#     # print(kmeans.cluster_centers_)
#     return kmeans 

# estimator = showKmeans(seen_weights, [2, 5, 7], 3, 'q', 'i')

    

torch.Size([256, 24])


/home/zuser/anaconda3/envs/RF-Diffusion/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


(24,)


In [ ]:
# estimator.cluster_centers_.shape

(8, 256)

In [3]:
def merge(seen_weights, idxs, n_clusters=8):
    merged_weights = {}
    for key in seen_weights[0].keys():
        if 'lora_A' in key:
            keyA = key
            keyB = key.replace('lora_A', 'lora_B')
            weightsA = [seen_weights[i][keyA] for i in idxs]
            weightsB = [seen_weights[i][keyB] for i in idxs]
            weights = [torch.cat([weightsA[i].T, weightsB[i]], dim=0) for i in range(len(weightsA))]
            weights = torch.cat(weights, dim=1).T
            from sklearn.cluster import KMeans
            kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(weights.cpu().detach().numpy())
            centers = kmeans.cluster_centers_
            # norm process 1
            si_norm, _ = torch.max(weights, dim=1)
            avg_norms = torch.zeros(n_clusters)  # Since n_clusters=8
            for i in range(n_clusters):
                cluster_mask = (kmeans.labels_ == i)
                avg_norms[i] = torch.mean(si_norm[cluster_mask])
            centers = (avg_norms / centers.max(axis=1)).unsqueeze(1) * centers
            # norm process 2
            original_rank = len(weights) // len(idxs)
            merged_rank =  n_clusters
            centers = centers * (original_rank / merged_rank) ** 0.5
            # merge
            merged_weights[keyA] = centers[:, :centers.shape[1] // 2]
            merged_weights[keyB] = centers[:, centers.shape[1] // 2:].T
    return merged_weights, kmeans.labels_
merged_weights, labels = merge(seen_weights, [3, 5])
print(labels)

/home/zuser/anaconda3/envs/RF-Diffusion/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/zuser/anaconda3/envs/RF-Diffusion/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/zuser/anaconda3/envs/RF-Diffusion/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/zuser/anaconda3/envs/RF-Diffusion/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412:

[4 3 5 3 7 0 0 2 1 6 1 1 1 6 1 6]


In [4]:
torch.save(merged_weights, f'FT_Client4/lora/lora_weights.pt')